# Github link for data
https://github.com/CSSEGISandData/COVID-19

# Import Libraries

In [1]:
import numpy as np
import pandas as pd

# Import data(29th October, 2020)

In [2]:
# Data of 29th October, 2020
data = pd.read_csv("10-29-2020.csv")
# State ISO codes
US_state = pd.read_csv('USA_state_code.csv')
data.head(2)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-10-30 04:24:49,33.93911,67.709953,41268,1532,34239,5497.0,Afghanistan,106.010169,3.712319
1,NaN,NaN,NaN,Albania,2020-10-30 04:24:49,41.15330,20.168300,20315,499,11007,8809.0,Albania,705.921190,2.456313


In [3]:
US_state.head(2)

,code,state,category
0,AL,Alabama,state
1,AK,Alaska,state


## Keep the necessary columns

In [4]:
#Keep the needed columns
data = data.drop(['FIPS','Admin2','Last_Update','Lat','Long_','Combined_Key',
                     'Incidence_Rate', 'Case-Fatality_Ratio'],axis=1)
data.head(2)

,Province_State,Country_Region,Confirmed,Deaths,Recovered,Active
0,NaN,Afghanistan,41268,1532,34239,5497.0
1,NaN,Albania,20315,499,11007,8809.0


# Filter only the US data

In [5]:
US_country = data[data.Country_Region=="US"]
US_country.head(2)

,Province_State,Country_Region,Confirmed,Deaths,Recovered,Active
634,Alabama,US,2126,31,0,2095.0
635,Alabama,US,6768,69,0,6699.0


## Group the values according to Province_State

In [6]:
#Group the values according to Province_State
class_group = US_country[US_country.columns[2:]].groupby(US_country[US_country.columns[0]])

In [7]:
class_group.get_group('Alabama')

,Confirmed,Deaths,Recovered,Active
634,2126,31,0,2095.0
635,6768,69,0,6699.0
636,1055,9,0,1046.0
637,861,15,0,846.0
638,2009,25,0,1984.0
...,...,...,...,...
698,0,0,0,0.0
699,2874,93,0,2781.0
700,758,12,0,746.0
701,571,12,0,559.0


## Add the columns according to Province_State

In [8]:
#prepare the data containg the total sum according to province state
C = []
for i in class_group.groups:
    C.append([sum(class_group.get_group(i)['Confirmed']),sum(class_group.get_group(i)['Deaths']),
              sum(class_group.get_group(i)['Recovered']),sum(class_group.get_group(i)['Active'])])

#convert list to numpy array
C_array = np.asarray(C)
#C_array

In [9]:
#prepare dataframe 
state_data = pd.DataFrame(C_array,columns=['Confirmed','Deaths',
                               'Recovered','Active'])

# A new state column
state_data['State'] = class_group.groups
state_data.head(2)

,Confirmed,Deaths,Recovered,Active,State
0,189149.0,2914.0,0.0,186235.0,Alabama
1,15522.0,77.0,0.0,15445.0,Alaska


# Create a "code" column using the US_state dataset containing the ISO code of the states

In [10]:
# lowercase the state column
US_state["state"] = US_state["state"].str.lower()

In [11]:
#list to store country code
state_code = np.array([])
for i in state_data['State'].str.lower():
    if np.any(US_state["state"]==i):
        state_code = np.append(state_code,US_state[US_state['state']==i]['code'])
    else:
        state_code = np.append(state_code,0)
 

In [12]:
#state_code

In [13]:
state_data['code'] = state_code
state_data.head()

,Confirmed,Deaths,Recovered,Active,State,code
0,189149.0,2914.0,0.0,186235.0,Alabama,AL
1,15522.0,77.0,0.0,15445.0,Alaska,AK
2,242480.0,5918.0,0.0,NaN,Arizona,AZ
3,109712.0,1894.0,0.0,107818.0,Arkansas,AR
4,923648.0,17559.0,0.0,906089.0,California,CA


# Plotting

In [14]:
import chart_studio.plotly as py
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [15]:
data = dict(type = 'choropleth',
            locations = state_data['code'],
            locationmode = 'USA-states',
            colorscale= 'Portland',
            text= state_data['State'],
            z=state_data['Confirmed'],
            colorbar = {'title':'Confirmed'})

layout = dict(title = 'Corona Outbreak(as of 29/10/2020)',geo = {'scope':'usa'})
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap)